<a href="https://colab.research.google.com/github/RodolfoFigueroa/madi2023/blob/main/Unidad_2/03_Algoritmos_de_ordenamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En esta libreta, veremos algunos algoritmos de ordenamiento. 

Primero, importamos las librerías necesarias:

In [4]:
import numpy as np

Luego, generamos una lista de ejemplo:

In [8]:
L = np.random.randint(0, 10, 10)
print(L)

[9 8 6 4 4 2 6 2 6 5]


# Algoritmos

## Insertion sort

Para cada elemento $X$ de la lista (excluyendo el primero), hacemos lo siguiente:

1. Lo comparamos con el elemento a su izquierda, llámese $L$.
2. Si $X < L$, intercambiamos $X$ y $L$.
3. Repetimos hasta que $L < X$, o lleguemos al principio de la lista.

In [11]:
def insertion_sort(a):
    for i in range(1, len(a)):
        key = a[i]
        j = i - 1
        while j >= 0 and key < a[j]:
            a[j + 1] = a[j]
            j = j - 1
        a[j + 1] = key

In [ ]:
temp = L.copy()
insertion_sort(temp)
print(temp)

* **Tiempo:** En el peor de los casos, el arreglo está ordenado al revés. Entonces, tendremos que comparar cada elemento con todos los elementos anteriores; en otras palabras, en la primera iteración haremos 1 comparación, luego 2, luego 3, y así hasta $n$. Por lo tanto, el número total de comparaciones es igual a la suma de los primeros $n$ naturales, que sabemos es de orden $O(n^2)$.
* **Espacio:** Como solo maneja variables simples, la complejidad es $O(1)$.

## Bubble sort

1. Recorremos el arreglo, comparando cada par de elementos.
2. Si un par no está ordenado, los intercambiamos.
3. Repetimos esto hasta llegar al final del arreglo.
4. Si hicimos al menos un intercambio (i.e., la lista no estaba ordenada), volvemos a empezar.

In [30]:
def bubble_sort(a):
    swapped = True
    while swapped:
        swapped = False
        for i in range(len(a)-1):
            if a[i] > a[i+1]:
                a[i], a[i+1] = a[i+1], a[i]
                swapped = True

In [31]:
temp = L.copy()
bubble_sort(temp)
print(temp)

[2 2 4 4 5 6 6 6 8 9]


* **Tiempo:** Si el arreglo está ordenado al revés, en la primera iteración "llevaremos" el elemento más grande al último lugar. Después, en la siguiente iteración haremos lo mismo con el segundo más grande, y así sucesivamente. Entonces, la complejidad será de $O(n^2)$.
* **Espacio:** Como solo maneja variables simples, la complejidad es $O(1)$.

Podemos mejorar este algoritmo notando que, en la $n$-ésima iteración, estamos colocando el $n$-ésimo elemento más grande en su lugar final. Entonces, no tenemos que revisarlo en iteraciones subsecuentes:

In [32]:
def bubble_sort_optim(a):
    swapped = True
    n = len(a)
    while swapped:
        swapped = False
        for i in range(n-1):
            if a[i] > a[i+1]:
                a[i], a[i+1] = a[i+1], a[i]
                swapped = True
        n -= 1

In [33]:
temp = L.copy()
bubble_sort_optim(temp)
print(temp)

[2 2 4 4 5 6 6 6 8 9]


Por desgracia, esto no cambia la complejidad en el peor de los casos.

## Merge sort

1. Partimos la lista a la mitad, en parte izquierda (`L`) y derecha (`R`):
2. Ordenamos las dos partes de manera recursiva.
3. *Combinamos* las partes ordenadas en la lista final.

El último paso lo logramos a través de la función `merge(L, R)`. Dadas las dos mitades de la lista, esta hace lo siguiente:
1. Inicializa una lista vacía `out`.
2. Compara el primer elemento de `L` con el primero de `R`, y añade el más pequeño a `out`.
3. Repite esto hasta que una de las listas esté vacía.
4. Añade todos los elementos restantes de la lista no vacía a `out`.

In [34]:
def merge_sort(a):
    if len(a) <= 1:
        return a
    
    mid = len(a) // 2
    left = a[:mid]
    right = a[mid:]
    
    left = merge_sort(left)
    right = merge_sort(right)
    return merge(left, right)

def merge(left, right):
    out = []
    while len(left) > 0 and len(right) > 0:
        if left[0] <= right[0]:
            out.append(left.pop(0))
        else:
            out.append(right.pop(0))
    
    out += left
    out += right
    return out

In [37]:
temp = list(L.copy())
temp = merge_sort(temp)
print(temp)

[2, 2, 4, 4, 5, 6, 6, 6, 8, 9]


**Tiempo** 

Notemos que el tiempo de ejecución sigue la siguiente relación de recurrencia:

$$
T(n) = 2T\left(\frac{n}{2}\right) + n
$$

El primer término corresponde a ordenar las dos mitades de la lista de manera recursiva, mientras que el segundo es el ocupado por la función `merge`.

Podemos obtener una expresión cerrada utilizando el [teorema maestro](https://en.wikipedia.org/wiki/Master_theorem_(analysis_of_algorithms)). Primero, el exponente crítico es:

$$
c_\text{crit}=\log_2(2)=1
$$

Dado que $f(n) = n = \Theta(n^{c_\text{crit}}\log^k n)$, con $k=0$, concluimos que estamos en el caso 2 de la tabla. Por lo tanto:

$$
T(n) = \Theta(n\log n)
$$

**Espacio**

Dado que partimos la lista en 2 recursivamente hasta llegar a una lista con 1 o 0 elementos, el tamaño máximo de la pila de llamada será de $O(\log n)$. Por otro lado, en la función `merge` tenemos el arreglo `out` al cual vamos añadiendo los elementos ordenados. Al finalizar la ejecución, este tendrá tamaño $O(n)$.

Por lo tanto, la complejidad en espacio es $O(\log n) + O(n) \sim O(n)$.

## Counting sort

1. Calculamos el máximo y mínimo del arreglo (`max` y `min`). Para nuestro arreglo de ejemplo, `min=2` y `max=9`.
2. Generamos un arreglo auxiliar `counts` de tamaño `max - min + 1`. La `i`-ésima entrada corresponde a cuántas veces está el elemento `i + min` en el arreglo a ordenar.

En nuestro ejemplo, este arreglo se ve de la siguiente forma:

```
counts = [2, 0, 2, 1, 3, 0, 1, 1]
```

Es decir, el `2` aparece 2 veces, el `3` 0 veces, etc.

3. Sumamos el elemento `i` del arreglo `counts` con el `i+1`, y lo guardamos en la posición `i`. Después de hacer esto para cada par de elementos, la entrada `i` nos dice *hasta dónde llegan* el elemento `i + min` de la lista a ordenar.

Aplicándolo a nuestro ejemplo:

```
counts = [2, 2, 4, 5, 8, 8, 9, 10]
```

* La primera entrada nos dice que el elemento `2` de la lista va en las posiciones 0 y 1 de la lista ordenada.
* La segunda entrada nos dice que el elemento `3` no va en ninguna posición.
* La tercera nos dice que el elemento `4` va en las posiciones 3 y 4.
* Luego, el elemento `5` van en la posición 5.
* Etcétera

In [69]:
def counting_sort(a):
    min_ = min(a)
    max_ = max(a)
    k = max_ - min_ + 1
    
    count = [0] * k
    out = [None] * len(a)
    
    for i in range(len(a)):
        key = a[i] - min_
        count[key] += 1

    for i in range(1, k):
        count[i] += count[i-1]
   
    for i in range(len(a)-1, -1, -1):
        key = a[i] - min_
        count[key] -= 1
        out[count[key]] = a[i]
    
    return out

In [71]:
temp = L.copy()
temp = counting_sort(temp)
print(temp)

[2, 0, 2, 1, 3, 0, 1, 1]
[2, 2, 4, 5, 8, 8, 9, 10]
[2, 2, 4, 4, 5, 6, 6, 6, 8, 9]


* **Tiempo:** Analizando los bucles *for*, vemos que la complejidad es $O(n) + O(k) + O(n) \sim O(k+n)$
* **Espacio:** Dado que la lista `count` tiene tamaño $k$, y la lista `out` $n$, se concluye que la complejidad es $O(k+n)$.

# Ejercicios